# Fuse all Data together

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
action_name_for_file = ['still', 'nod', 'shake', 'tilt', 'eyes-ud', 'eyes-lr', 'brow-raiser', 'brow-lowerer', 'wink-r',
                        'wink-l', 'lip-puller', 'chin-raiser', 'mouth-stretch', 'chewing', 'speaking', 'walking', 'running']

In [ ]:
def data_fusion(patient, right_ear, left_ear):
    '''
    Automate input reading: select patient, left/right ear
    Read in csv file, change to ms timestamps
    '''
    # Check earside
    ear_side = 'right' if right_ear else ('left' if left_ear else None)
    if ear_side is None:
        raise ValueError("Either `right_ear` or `left_ear` must be specified.")
    
    # IMU
    df_data_imu = pd.DataFrame()

    for action in action_name_for_file:
        file_path = f'../data/Dataset/P{patient}/EARBUDS/{patient}-{action}-imu-{ear_side}.csv'
        
        # Reading the CSV file
        try:
            df_temp = pd.read_csv(
                file_path,
                sep=',',           # specify delimiter (default is ',')
                header=0,          # row number to use as column names (0 means the first row)
                na_values=['NA', '']  # specify which values should be considered NaN
            )
            df_data_imu = pd.concat([df_data_imu, df_temp], ignore_index=True)
            #print(f"File found: {file_path}")
        
        except FileNotFoundError:
            print(f"File not found: {file_path}")

    df_data_imu *= [1, 0.061, 0.061, 0.061, 17.5, 17.5, 17.5]
    df_data_imu['timestamp'] = pd.to_datetime(df_data_imu['timestamp'], unit='ms')
    df_data_imu['timestamp_diff'] = df_data_imu['timestamp'].diff()

    ## PPG 
    df_data_ppg = pd.DataFrame()

    for action in action_name_for_file:
        file_path = f'../data/Dataset/P{patient}/EARBUDS/{patient}-{action}-ppg-{ear_side}.csv'
        
        # Reading the CSV file
        try:
            df_temp = pd.read_csv(
                file_path,
                sep=',',           # specify delimiter (default is ',')
                header=0,          # row number to use as column names (0 means the first row)
                na_values=['NA', '']  # specify which values should be considered NaN
            )
            df_data_ppg = pd.concat([df_data_ppg, df_temp], ignore_index=True)
            #print(f"File found: {file_path}")
        
        except FileNotFoundError:
            print(f"File not found: {file_path}")

    # Handle PPG signals: Add a 'configuration' column initialized with NaN values
    df_data_ppg['sensor_config'] = pd.NA

    # Start with adding sensor configuration index
    config_type = 1
    config_rows = []

    # Loop through rows, change timesteps, store configuration changes and assign configuration index
    for i, row in df_data_ppg.iterrows():
        if row['timestamp'].startswith('#'):
            df_data_ppg.at[i, 'sensor_config'] = 'Config Change'
            config_rows.append(i)
            config_type += 1
        else:
            df_data_ppg.at[i, 'sensor_config'] = config_type
            df_data_ppg.at[i, 'timestamp'] = pd.to_datetime(df_data_ppg.at[i, 'timestamp'], unit='ms')
            #df_data_ppg['timestamp_diff'] = df_data_ppg['timestamp'].diff()

    df_data_zephyr = pd.read_csv(
        '../data/Dataset/P'+ str(patient) + '/ZEPHYR/' + str(patient) + '_' + 'Summary.csv',
        sep=',',           # specify delimiter (default is ',')
        header=0,          # row number to use as column names (0 means the first row)
        na_values=['NA', ''],  # specify which values should be considered NaN
    )
    # Change timesteps
    df_data_zephyr['Timestamp'] = pd.to_datetime(df_data_zephyr['Timestamp'], unit='ms')

    return df_data_imu, df_data_ppg, df_data_zephyr

In [ ]:
overview = pd.DataFrame(columns=['Patient', 'Start time actions PPG', 'Stop Time PPG', 'Start time Zephyr', 'Stop time Zephyr'])

for patient in range(0, 28):
    df_data_imu, df_data_ppg, df_data_zephyr = data_fusion(patient, right_ear=False, left_ear=True)
    overview.loc[len(overview)] = [patient, df_data_ppg['timestamp'][0], df_data_ppg['timestamp'][len(df_data_ppg['timestamp'])-1], df_data_zephyr['Timestamp'][0], df_data_zephyr['Timestamp'][len(df_data_zephyr['Timestamp'])-1] ]



In [ ]:
overview

In [ ]:
df_data_imu, df_data_ppg, df_data_zephyr = data_fusion(patient=1, right_ear=False, left_ear=True)

In [ ]:
df_data_ppg

In [ ]:
df_data_imu

In [ ]:
print(df_data_imu.to_string())

In [ ]:
df_data_zephyr

In [ ]:
# Plot the IMU Timestamps
plt.figure(figsize=(20, 12))
plt.plot(df_data_imu['timestamp_diff'][1:], color='b', label="IMU Timestamps",)
plt.title("All Timestamps differences of the IMU signals for one patient and all actions")
plt.xlabel("Timestamp samples")
plt.ylabel("Time difference of Timestamps")
plt.legend()
plt.tight_layout()

#plt.savefig('../results/data_analysis/tdiff_imu_p0.png')

plt.show()

In [ ]:
# Plot the IMU Timestamps
plt.figure(figsize=(20, 12))
plt.plot(df_data_imu['timestamp'], color='b', label="IMU Timestamps")
plt.plot(df_data_zephyr['Timestamp'], color='r', label='Zephyr')
plt.title("All Timestamps of the IMU signals for one patient and all actions")
plt.xlabel("Timestamp samples")
plt.ylabel("Absolut time - (day, min)")
plt.legend()
plt.tight_layout()

#plt.savefig('../results/data_analysis/t_abs_imu_p0.png')

plt.show()